#aqui va el script del problema 3
#AQUI VA EL SCRIPT DEL EJERCICIO 3
#AQUI
# Script para el Ejercicio 3: Comparación Pandas vs PySpark con el dataset AirQualityUCI
# 1 Importación de Librerias
import pandas as pd
import numpy as np
#  2Ruta del Archivo
#la 'r' al inicio es importante en Windows/SEGUN el video, se recomienda eso, igual puedo poner que recise la existencia para luego descargar 
CSV_FILE = r'C:\Users\diego\Desktop\PROYECTO_\datos\ejer_3\AirQualityUCI.csv'
# 3 Funciones de Procesamiento
def limpiar_data(df):
    """
    Realiza una limpieza exhaustiva del DataFrame de Calidad del Aire.
    - Elimina columnas y filas vacías.
    - Reemplaza el marcador de nulos (-200) por NaN.
    - Convierte las columnas de fecha y hora a un índice de tipo datetime.
    """
    print("[Limpieza] Inciando limpieza de datos...")
    #Eliminar las dos columnas(creo que estan vacias)
    df = df.iloc[:, :-2]
    #Eliminar filas donde todos los valores son nulo
    df.dropna(how='all', inplace=True)
    # Reemplazar el valor -200 (marcador de nulos específico de este dataset) con NaN de numpy
    df.replace(to_replace=-200, value=np.nan, inplace=True)
    # Combinar Date y Time en una sola columna de tipo datetime para análisis de series de tiempo
    df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H.%M.%S')
    df.set_index('DateTime', inplace=True)
    df.drop(['Date', 'Time'], axis=1, inplace=True)
    print("[Limpieza] Limpieza completada.")
    return df
def _pandas(df):
    """
    Realiza las operaciones de filtrado, agrupamiento e imputación con Pandas
    y muestra los resultados en la consola.
    """
    print("\n" + "="*50)
    print("ANALISIS CON PANDAS")
    print("="*50)
    # 1 filtro que puede servir: dias con temperatura mayor a 25C
    print("\n[PANDAS] 1. Filtrando registros con Temperatura > 25C...")
    dias_calurosos = df[df['T'] > 25]
    print(f"Se encontraron {len(dias_calurosos)} registros/horas con T > 25°C.")
    print("Mostrando los 5 primeros registros de días calurosos:")
    # .to_string() asegura que se vea bien en la terminal o que se vea, n0 logro que se vea bien 
    print(dias_calurosos[['T', 'RH', 'AH']].head().to_string())
    print("-" * 50)
    # 2 calcular y mostrar la media de tem  diaria? o de algun lugar especifico (benceno (C6H6)(porque este???))
    print("\n[PANDAS] 2. Calculando la media diaria de Benceno (C6H6(GT))...")
    # .resample('D') es una poderosa función para agrupar por día
    media_diaria_benceno = df['C6H6(GT)'].resample('D').mean()
    print("Mostrando la media de los últimos 5 días con registros:")
    print(media_diaria_benceno.dropna().tail().to_string())
    print("-" * 50)
    # 3 Benceno(no encerio porque este?)mostramos la media, mediana y el metodo ffill para hacer algo con los nulos, deja buscar que es ffill
    # ffill es fill forward, osea que llena con el ultimo valor conocido, osea que si hay nulos seguidos, los llena con el ultimo valor que no es nulo
    #aver que resulta(no le tengo mucha fe(lol))
    print("\n[PANDAS] 3 realizando imputación en la columna 'C6H6(GT)'...")
    columna_benceno = df[['C6H6(GT)']].copy()
    columna_benceno['C6H6_con_media'] = columna_benceno['C6H6(GT)'].fillna(columna_benceno['C6H6(GT)'].mean())
    columna_benceno['C6H6_con_mediana'] = columna_benceno['C6H6(GT)'].fillna(columna_benceno['C6H6(GT)'].median())
    columna_benceno['C6H6_con_ffill'] = columna_benceno['C6H6(GT)'].fillna(method='ffill')
    print("\nVerificación de imputación (mostrando 5 filas donde el valor original era nulo):")
    print(columna_benceno[columna_benceno['C6H6(GT)'].isnull()].head().to_string())
#dejo esto o lo paso al java?/igual lo dejo expresado en el java(tengo muchas dudas si se puede java y python en el mismo proyecto)
def _pyspark():
    """
    como seria, muestra de lo que seria el codigo en pyspark
    """
    print("\n" + "="*50)
    print("EJEMPLO DE CÓDIGO(compraracion)")
    print("="*50)
    
    codigo_pyspark = """
# Para ejecutar esto, se necesita una instal de Java y Spark.(por ahora eso es lo necesario)
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, avg, to_date
# spark = SparkSession.builder.appName("AirQualitySpark").getOrCreate()

# # lectura del csv para el spark.manejo de decimales con ','
# # puede requerir filtrar data.(segun la guia(video))
# df_spark = spark.read.csv(
#     r'C:\\Users\\diego\\Desktop\\PROYECTO_\\datos\\ejer_3\\AirQualityUCI.csv',
#     header=True,
#     sep=';',
#     inferSchema=True
# ).replace(-200, None) # Reemplazar -200 por nulos

# # convertir columna de fecha y agrupar /utilizar la misma locacion de benceno 
# df_spark = df_spark.withColumn('Date', to_date(col('Date'), 'dd/MM/yyyy'))
# media_diaria_benceno_spark = df_spark.groupBy('Date').agg(avg('C6H6(GT)').alias('media_benceno'))

# print("Resultado de PySpark (media diaria de Benceno):")
# media_diaria_benceno_spark.orderBy('Date', ascending=False).show(5)
# spark.stop()
    """
    print(codigo_pyspark)
#4 Ejecución!(esta bien escrito/creo)
def main():
    """
    Funcin principal que orquesta la ejecución del script.
    """
    try:
        # Carga de datos especificos 
        print(f"Cargando dataset desde: {CSV_FILE}...")
        air_df = pd.read_csv(CSV_FILE, sep=';', decimal=',')
        # Limpieza de basura
        air_df_limpio = limpiar_data(air_df)
        print(f"Dimensiones del dataset limpio: {air_df_limpio.shape}")
        # se hace analisis
        _pandas(air_df_limpio)
        # Mostrar el código de ejemplo de PySpark
        _pyspark()
    except FileNotFoundError:
        print(f"error: No se encontro el archivo.")
        print(f"Ruta: '{CSV_FILE}'")
        print("Por favor, verifica que la ruta sea correcta y que el archivo exista.")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
if __name__ == "__main__":
    main()
    print("\n--- Script Finalizado ---")

# Ejercicio 3: Comparación Pandas vs PySpark
carga, limpia y analiza el dataset `AirQualityUCI` utilizando Pandas. Al final, muestra un ejemplo de cómo se realizarían operaciones similares en PySpark.

In [ ]:
import pandas as pd
import numpy as np
import os

# --- Configuración de la ruta del archivo ---
# ¡IMPORTANTE! Asegúrate de que esta ruta sea correcta en tu sistema.
URL="https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"

CSV_FILE = r'C:\Users\diego\Desktop\PROYECTO_\datos\ejer_3\AirQualityUCI.csv'
# Verificamos si el archivo existe para evitar errores
if not os.path.exists(CSV_FILE):
    print(f" ERROR: No se pudo encontrar el archivo en la ruta:\n{CSV_FILE}")
else:
    print(f" Archivo encontrado en:\n{CSV_FILE}")

 Archivo encontrado en:
C:\Users\diego\Desktop\PROYECTO_\datos\ejer_3\AirQualityUCI.csv


# definimos las fuciones de procesamiento y/o filtracion
 

In [ ]:
def limpiar_data(df):
    """
    Realiza una limpieza exhaustiva del DataFrame de Calidad del Aire.
    - Elimina columnas y filas vacías.
    - Reemplaza el marcador de nulos (-200) por NaN.
    - Convierte las columnas de fecha y hora a un índice de tipo datetime.
    """
    print("[Limpieza] Iniciando limpieza de datos...")
    
    # Eliminar las dos últimas columnas que suelen estar vacías
    df_cleaned = df.iloc[:, :-2].copy()
    
    # Eliminar filas donde todos los valores son nulos
    df_cleaned.dropna(how='all', inplace=True)
    
    # Reemplazar el valor -200 con NaN de numpy
    df_cleaned.replace(to_replace=-200, value=np.nan, inplace=True)
    
    # Combinar Date y Time en una sola columna de tipo datetime
    df_cleaned['DateTime'] = pd.to_datetime(df_cleaned['Date'] + ' ' + df_cleaned['Time'], format='%d/%m/%Y %H.%M.%S', errors='coerce')
    df_cleaned.set_index('DateTime', inplace=True)
    df_cleaned.drop(['Date', 'Time'], axis=1, inplace=True)
    
    print("[Limpieza] Limpieza completada.")
    return df_cleaned

def analisis_pandas(df):
    """
    Realiza las operaciones de filtrado, agrupamiento e imputación con Pandas.
    """
    print("\n" + "="*50)
    print("ANÁLISIS CON PANDAS")
    print("="*50)

    # 1. Filtrar registros con Temperatura > 25°C
    print("\n[PANDAS] 1. Filtrando registros con Temperatura > 25°C...")
    dias_calurosos = df[df['T'] > 25]
    print(f"Se encontraron {len(dias_calurosos)} registros/horas con T > 25°C.")
    print("Mostrando los 5 primeros registros de días calurosos:")
    display(dias_calurosos[['T', 'RH', 'AH']].head())
    print("-" * 50)

    # 2. Calcular la media diaria de Benceno
    print("\n[PANDAS] 2. Calculando la media diaria de Benceno (C6H6(GT))...")
    media_diaria_benceno = df['C6H6(GT)'].resample('D').mean()
    print("Mostrando la media de los últimos 5 días con registros:")
    display(media_diaria_benceno.dropna().tail())
    print("-" * 50)

    # 3. Realizar imputación en la columna 'C6H6(GT)'
    print("\n[PANDAS] 3. Realizando imputación en la columna 'C6H6(GT)'...")
    columna_benceno = df[['C6H6(GT)']].copy()
    columna_benceno['C6H6_con_media'] = columna_benceno['C6H6(GT)'].fillna(columna_benceno['C6H6(GT)'].mean())
    columna_benceno['C6H6_con_mediana'] = columna_benceno['C6H6(GT)'].fillna(columna_benceno['C6H6(GT)'].median())
    columna_benceno['C6H6_con_ffill'] = columna_benceno['C6H6(GT)'].fillna(method='ffill')
    print("\nVerificación de imputación (mostrando 5 filas donde el valor original era nulo):")
    display(columna_benceno[columna_benceno['C6H6(GT)'].isnull()].head())

# Cargamos datos

In [ ]:
try:
    print(f"Cargando dataset desde: {CSV_FILE}...")
    air_df = pd.read_csv(CSV_FILE, sep=';', decimal=',')
    
    # Limpieza de datos
    air_df_limpio = limpiar_data(air_df)
    
    print(f"\nDimensiones del dataset limpio: {air_df_limpio.shape}")
    print("Primeras 5 filas del dataset limpio:")
    display(air_df_limpio.head())
    
except FileNotFoundError:
    print(f" ERROR: No se encontró el archivo en la ruta '{CSV_FILE}'")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Cargando dataset desde: C:\Users\diego\Desktop\PROYECTO_\datos\ejer_3\AirQualityUCI.csv...
[Limpieza] Iniciando limpieza de datos...
[Limpieza] Limpieza completada.

Dimensiones del dataset limpio: (9357, 13)
Primeras 5 filas del dataset limpio:


,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
DateTime,,,,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
2004-03-10 19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2004-03-10 20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
2004-03-10 21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
2004-03-10 22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


# con pandas

In [ ]:
# Se ejecuta el análisis solo si el DataFrame 'air_df_limpio' se creó correctamente
if 'air_df_limpio' in locals():
    analisis_pandas(air_df_limpio)
else:
    print("El DataFrame 'air_df_limpio' no fue creado. Por favor, revisa la celda anterior.")


ANÁLISIS CON PANDAS

[PANDAS] 1. Filtrando registros con Temperatura > 25°C...
Se encontraron 2059 registros/horas con T > 25°C.
Mostrando los 5 primeros registros de días calurosos:


,T,RH,AH
DateTime,,,
2004-03-16 13:00:00,25.3,26.1,0.8264
2004-03-16 14:00:00,25.8,23.2,0.7589
2004-03-16 15:00:00,27.0,20.2,0.7094
2004-03-16 16:00:00,28.2,18.6,0.7014
2004-03-16 17:00:00,28.0,19.1,0.7098


--------------------------------------------------

[PANDAS] 2. Calculando la media diaria de Benceno (C6H6(GT))...
Mostrando la media de los últimos 5 días con registros:


DateTime
2005-03-31    5.220833
2005-04-01    3.412500
2005-04-02    2.529167
2005-04-03    4.316667
2005-04-04    8.440000
Name: C6H6(GT), dtype: float64

--------------------------------------------------

[PANDAS] 3. Realizando imputación en la columna 'C6H6(GT)'...

Verificación de imputación (mostrando 5 filas donde el valor original era nulo):


C:\Users\diego\AppData\Local\Temp\ipykernel_14724\3703551525.py:55: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  columna_benceno['C6H6_con_ffill'] = columna_benceno['C6H6(GT)'].fillna(method='ffill')


,C6H6(GT),C6H6_con_media,C6H6_con_mediana,C6H6_con_ffill
DateTime,,,,
2004-04-01 14:00:00,NaN,10.083105,8.2,8.6
2004-04-01 15:00:00,NaN,10.083105,8.2,8.6
2004-04-01 16:00:00,NaN,10.083105,8.2,8.6
2004-04-08 23:00:00,NaN,10.083105,8.2,6.3
2004-04-09 00:00:00,NaN,10.083105,8.2,6.3


# pyspark
 por problemas con java y no saber configurarlo bien no lo tengo ejecutable